## Chapter 15: Production Readiness and Day 2 Operations

Deploying your application to the cloud is a major milestone, but it’s only the beginning. Once your application is live, you enter the world of **Day 2 operations**—keeping it running smoothly, monitoring its health, responding to incidents, scaling to meet demand, and managing secrets securely. In this chapter, we’ll take the e‑commerce application we deployed to Azure Container Apps and make it production‑ready. You’ll learn how to:

- Ship logs and metrics to Azure Monitor and Application Insights for centralized observability.
- Configure auto‑scaling rules so your services can handle load and save costs.
- Move secrets from environment variables to Azure Key Vault, using managed identities for secure access.
- Set up distributed tracing to debug cross‑service issues.
- Implement best practices for health checks and readiness probes.

By the end, your application will be robust, observable, and ready for real users.

---

### 15.1 The Gap Between Development and Production

In development, you rely on the Aspire Dashboard for logs, traces, and metrics. It’s a fantastic tool for the inner loop, but it’s not designed for production. In production, you need:

- **Long‑term retention** of logs and metrics.
- **Centralized aggregation** from all instances and services.
- **Alerting** on anomalies (e.g., high error rate, low disk space).
- **Role‑based access control** for who can view data.
- **Integration with incident management** tools.

Azure provides a suite of monitoring services: **Log Analytics** for log storage and querying, **Application Insights** for application performance monitoring (APM), and **Azure Monitor** for metrics and alerts. These services integrate seamlessly with .NET and with Azure Container Apps.

---

### 15.2 Logging and Monitoring in Production

#### 15.2.1 Shipping Logs to Azure Log Analytics

Azure Container Apps can automatically send container stdout/stderr logs to a Log Analytics workspace. When you provision an ACA environment, you can link it to a Log Analytics workspace. In our `azd` deployment, this is already set up.

To view logs:

1. In the Azure portal, navigate to your Container App (e.g., `apiservice`).
2. Under **Monitoring**, select **Logs**.
3. Run a query like:

   ```kusto
   ContainerAppConsoleLogs_CL
   | where ContainerAppName_s == "apiservice"
   | project TimeGenerated, Log_s
   | order by TimeGenerated desc
   ```

This shows raw console logs. However, structured logs (like those from `ILogger`) are even better. Our services use `builder.AddServiceDefaults()`, which configures OpenTelemetry logging. To send structured logs to Log Analytics, we need to use an OpenTelemetry exporter that speaks to Log Analytics. The easiest way is to use Application Insights, which can ingest logs, traces, and metrics.

#### 15.2.2 Application Insights for Traces, Metrics, and Logs

Application Insights is a feature of Azure Monitor that provides powerful application performance monitoring. It can collect:

- **Requests**: incoming HTTP requests.
- **Dependencies**: calls to databases, HTTP services, queues.
- **Exceptions**: thrown exceptions.
- **Traces**: custom log messages.
- **Metrics**: custom and system metrics.

To enable Application Insights, we need to:

- Add the Application Insights SDK or the OpenTelemetry Azure Monitor exporter to each service.
- Configure the connection string via an environment variable.
- Ensure the exporter sends telemetry to Application Insights.

##### Step 1: Add the OpenTelemetry Azure Monitor Exporter

In each service project (API, Web, Worker), install the NuGet package:

```bash
dotnet add package Azure.Monitor.OpenTelemetry.Exporter
```

##### Step 2: Configure OpenTelemetry to Use Azure Monitor

In `Program.cs`, after `AddServiceDefaults()`, we need to modify the OpenTelemetry setup to add the Azure Monitor exporter. However, `AddServiceDefaults` already configures OpenTelemetry with an OTLP exporter (pointing to the Aspire Dashboard). We need to either replace that or add a second exporter. In production, we don’t want to send telemetry to the Aspire Dashboard (which isn’t running). So we can conditionally add the Azure Monitor exporter based on the environment.

In `Program.cs`:

```csharp
if (builder.Environment.IsProduction())
{
    builder.Services.AddOpenTelemetry()
        .UseAzureMonitor(options =>
        {
            options.ConnectionString = builder.Configuration["APPLICATIONINSIGHTS_CONNECTION_STRING"];
        });
}
else
{
    // In development, the defaults already send to Aspire Dashboard
}
```

But note: `AddServiceDefaults` already calls `AddOpenTelemetry` with metrics and tracing. We can’t call it again without interfering. The recommended approach is to modify the ServiceDefaults project to be more flexible, or to remove the default exporter and add our own. For simplicity, we'll conditionally clear the default exporters and add Azure Monitor.

In `Program.cs`, after `AddServiceDefaults`, we can do:

```csharp
builder.Services.Configure<OpenTelemetryLoggerOptions>(logging => logging.ClearExporters());
builder.Services.ConfigureOpenTelemetryMeterProvider(metrics => metrics.ClearExporters());
builder.Services.ConfigureOpenTelemetryTracerProvider(tracing => tracing.ClearExporters());

// Then add Azure Monitor
builder.Services.AddOpenTelemetry()
    .UseAzureMonitor(options =>
    {
        options.ConnectionString = builder.Configuration["APPLICATIONINSIGHTS_CONNECTION_STRING"];
    });
```

This is a bit hacky. A cleaner approach is to have a feature flag or environment variable that tells `AddServiceDefaults` to not add exporters, and then we add them manually. You could modify the ServiceDefaults project to accept a delegate for configuring exporters. We'll skip that complexity here.

##### Step 3: Set the Application Insights Connection String

In your ACA environment, you can set environment variables. With `azd`, you can add the connection string as a secret. First, create an Application Insights resource (you can add it to your Bicep). Then, set the environment variable `APPLICATIONINSIGHTS_CONNECTION_STRING` in the Container App definitions.

In Bicep, you might have:

```bicep
resource appInsights 'Microsoft.Insights/components@2020-02-02' = {
  name: '${resourceBaseName}-ai'
  location: location
  kind: 'web'
  properties: {
    Application_Type: 'web'
  }
}

// Then in Container App environment variables:
environmentVariables: [
  {
    name: 'APPLICATIONINSIGHTS_CONNECTION_STRING'
    value: appInsights.properties.ConnectionString
  }
]
```

`azd` can also automatically create an Application Insights instance and inject its connection string if you use the right patterns.

##### Step 4: Verify

After redeploying, go to the Application Insights resource in the Azure portal. You should see requests, dependencies, traces, and exceptions. You can also query logs using Kusto.

#### 15.2.3 Distributed Tracing with Application Insights

One of the most powerful features of Application Insights is **distributed tracing**. When a request flows from the web frontend to the API to the database, Application Insights correlates these operations into a single **transaction**. You can see the entire call stack, including dependencies, and pinpoint where time is spent.

Because we used `AddServiceDefaults`, our services already propagate trace context via W3C TraceContext. The Azure Monitor exporter will send this trace data, and Application Insights will stitch it together.

In the portal, go to Application Insights → **Transaction search** or **Performance** and click on an operation to see the end‑to‑end trace.

---

### 15.3 Configuring Auto‑Scaling in Azure Container Apps

Azure Container Apps supports several scaling rules:

- **HTTP scaling**: based on the number of concurrent HTTP requests.
- **Event‑driven scaling** (KEDA): based on queue length, Kafka offset, etc.
- **CPU/Memory scaling**: based on resource usage.
- **Custom scaling** (using KEDA scalers).

By default, a Container App scales to zero when idle (if you set `minReplicas` to 0). This is cost‑effective but may cause cold starts. You can adjust `minReplicas` and `maxReplicas` based on your needs.

#### 15.3.1 Adding HTTP Scaling Rules

In your Bicep or Container App configuration, you can define scaling rules. For example, to scale the API based on HTTP traffic:

```bicep
resource apiService 'Microsoft.App/containerApps@2023-05-01' = {
  // ...
  properties: {
    template: {
      // ...
      scale: {
        minReplicas: 0
        maxReplicas: 10
        rules: [
          {
            name: 'http-scale'
            http: {
              metadata: {
                concurrentRequests: '100'
              }
            }
          }
        ]
      }
    }
  }
}
```

This scales out when the number of concurrent HTTP requests exceeds 100 per replica.

#### 15.3.2 Event‑Driven Scaling for the Worker

Our worker service consumes from RabbitMQ. To scale it based on queue length, we need a KEDA scaler for RabbitMQ. First, we need to ensure RabbitMQ itself is accessible (if using RabbitMQ as a container in ACA, we need to expose it internally). Alternatively, if using Azure Service Bus, KEDA has built‑in support.

Assuming we have a RabbitMQ host, we can add a scaling rule:

```bicep
scale: {
  minReplicas: 0
  maxReplicas: 20
  rules: [
    {
      name: 'rabbitmq-queue'
      custom: {
        type: 'rabbitmq'
        metadata: {
          queueName: 'order-created-queue'
          host: 'rabbitmq.default.svc.cluster.local:5672' // if in same cluster
          protocol: 'amqp'
          mode: 'QueueLength'
          value: '10' // scale when queue length exceeds 10
        }
        auth: [
          // secrets reference if needed
        ]
      }
    }
  ]
}
```

This requires the RabbitMQ scaler to be available in the ACA environment (it is by default, as KEDA is pre‑installed). You also need to provide credentials via secrets.

#### 15.3.3 Setting Scaling in the AppHost (Aspire Way)

Aspire allows you to add scaling annotations that `azd` can translate into ACA scaling rules. For example:

```csharp
var workerService = builder.AddProject<Projects.MyAspireApp_WorkerService>("workerservice")
    .WithKedaScale(trigger: "rabbitmq", metadata: new Dictionary<string, string>
    {
        ["queueName"] = "order-created-queue",
        ["host"] = "rabbitmq:5672",
        ["value"] = "10"
    });
```

However, this feature is evolving. Check the latest Aspire documentation for `WithKedaScale` or similar.

For now, we'll manually edit the Bicep after `azd` generates it, or we can use `azd` hooks to modify the manifests.

---

### 15.4 Secrets Management with Azure Key Vault

In development, we used parameters and user secrets. In production, we should store secrets in Azure Key Vault and use managed identities to access them. ACA supports both direct secret references and integration with Key Vault via secrets.

#### 15.4.1 Creating a Key Vault and Storing Secrets

You can add a Key Vault resource to your Bicep:

```bicep
resource keyVault 'Microsoft.KeyVault/vaults@2022-07-01' = {
  name: '${resourceBaseName}-kv'
  location: location
  properties: {
    sku: { name: 'standard' }
    tenantId: subscription().tenantId
    accessPolicies: [
      // Grant access to the managed identity of your Container Apps
    ]
  }
}
```

To grant the Container App access, you need to enable managed identity on the Container App and then set an access policy.

#### 15.4.2 Using Managed Identity

Each Container App can have a system‑assigned or user‑assigned managed identity. When you enable it, Azure AD creates an identity for the app. You can then grant that identity access to Key Vault.

In Bicep:

```bicep
resource apiService 'Microsoft.App/containerApps@2023-05-01' = {
  identity: {
    type: 'SystemAssigned'
  }
  // ...
}
```

Then, in the Key Vault access policies:

```bicep
resource keyVault 'Microsoft.KeyVault/vaults@2022-07-01' existing = {
  name: kvName
}

resource keyVaultAccessPolicy 'Microsoft.KeyVault/vaults/accessPolicies@2022-07-01' = {
  parent: keyVault
  name: 'add'
  properties: {
    accessPolicies: [
      {
        tenantId: subscription().tenantId
        objectId: apiService.identity.principalId
        permissions: {
          secrets: ['get', 'list']
        }
      }
    ]
  }
}
```

#### 15.4.3 Referencing Secrets in Container Apps

In your Container App definition, you can reference secrets from Key Vault either by directly specifying the secret identifier and relying on the app's managed identity, or by pulling them into Container App secrets.

The simpler approach is to use **Container App secrets** that are backed by Key Vault. You define a secret in the Container App that references a Key Vault secret, and the system automatically retrieves it.

In Bicep:

```bicep
resource apiService 'Microsoft.App/containerApps@2023-05-01' = {
  properties: {
    configuration: {
      secrets: [
        {
          name: 'postgres-password'
          keyVaultUrl: 'https://myvault.vault.azure.net/secrets/postgres-password'
          identity: apiService.identity.principalId // or use a reference to the identity resource
        }
      ]
    }
    template: {
      containers: [
        {
          env: [
            {
              name: 'ConnectionStrings__productsdb'
              secretRef: 'postgres-password' // combine with other parts
            }
          ]
        }
      ]
    }
  }
}
```

But you need to build the full connection string. Usually, you'd have the password in Key Vault and the host/username in environment variables. You could store the entire connection string in Key Vault.

#### 15.4.4 Passing Secrets to the Application

In the application code, you don't need to change anything if you're using the connection string from configuration. The environment variable `ConnectionStrings__productsdb` will contain the full connection string (including password). If the password comes from Key Vault, it will be injected at runtime.

---

### 15.5 Hands‑on: Production‑Ready Enhancements

Now let's apply these concepts to our deployed e‑commerce application.

#### Step 1: Add Application Insights

1. Add the `Azure.Monitor.OpenTelemetry.Exporter` package to all service projects.
2. Modify each `Program.cs` to conditionally use Azure Monitor in production, as described.
3. Update the Bicep (or `azd` infra) to include an Application Insights resource.
4. Set the environment variable `APPLICATIONINSIGHTS_CONNECTION_STRING` for each Container App, referencing the resource.

#### Step 2: Configure Scaling

1. In the Bicep for the API service, add an HTTP scaling rule with `concurrentRequests: 50`.
2. For the worker, add a KEDA RabbitMQ scaling rule (if you have RabbitMQ). You'll need to ensure the worker has the RabbitMQ connection string as a secret, and that the scaler can access RabbitMQ. Since RabbitMQ is also running in ACA, you can use its internal service name.

#### Step 3: Move Secrets to Key Vault

1. Add a Key Vault resource to the Bicep.
2. Enable system‑assigned managed identity on each Container App.
3. Grant those identities `get` and `list` permissions to Key Vault secrets.
4. Move the PostgreSQL password and the RabbitMQ password to Key Vault secrets.
5. In the Container App configuration, reference these secrets using `keyVaultUrl` and `identity`.
6. Update the environment variables to use the secrets.

#### Step 4: Redeploy with `azd up`

Run `azd up` again. It will update the infrastructure and redeploy the containers.

#### Step 5: Verify

- Go to Application Insights and see that telemetry is flowing.
- Load test the API (e.g., using Apache Bench) to trigger scaling. Watch the replica count increase in the Azure portal.
- Ensure the application still works with secrets from Key Vault.

---

### 15.6 Summary

In this chapter, we transformed our development‑focused Aspire application into a production‑ready system. You learned:

- How to ship logs, traces, and metrics to Azure Monitor and Application Insights.
- How to configure auto‑scaling in Azure Container Apps for both HTTP and event‑driven workloads.
- How to manage secrets securely using Azure Key Vault and managed identities.
- The importance of observability, scaling, and security as part of Day 2 operations.

With these practices, your application is equipped to handle real users, scale on demand, and remain observable. In the final chapter, we’ll look at **Real‑World Project Structure and Best Practices**, including organizing large Aspire solutions, versioning strategies, and CI/CD pipelines.

---

**Exercises**

1. Set up a dashboard in Azure Monitor Workbooks to visualize key metrics (request rate, error rate, queue length).
2. Configure an alert rule that sends an email when the error rate exceeds 5% over 5 minutes.
3. Implement a health check endpoint that verifies connectivity to the database and Key Vault, and configure ACA to use it for liveness/readiness probes.
4. Rotate a secret in Key Vault and verify that the application picks up the new value without restart (if using secret references, it should be dynamic).

In Chapter 16, we’ll explore **Real‑World Project Structure and Best Practices** to help you organize and maintain large‑scale Aspire applications.